# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, silhouette_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats.mstats import winsorize
from sklearn.decomposition import PCA

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dicoding/Submission Membangun Proyek Machine Learning/clustering_results.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Nilai_Ujian,Jam_Belajar_Mingguan,Kehadiran,Ekstrakurikuler,Kelompok_Belajar,Nilai_Ujian_Binned,Jam_Belajar_Mingguan_Binned,Kehadiran_Binned,cluster,cluster_selected
0,70.0,23.0,89.0,Tidak,Besar,Rendah,Sedang,Sedang,1,1
1,70.0,21.0,99.0,Tidak,Kecil,Rendah,Sedang,Tinggi,1,1
2,94.0,3.0,70.0,Ya,Kecil,Tinggi,Rendah,Rendah,0,0
3,70.0,20.0,100.0,Tidak,Tidak,Rendah,Sedang,Tinggi,1,1
4,94.0,7.0,100.0,Ya,Tidak,Tinggi,Rendah,Tinggi,0,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [ ]:
# Pisahkan fitur (X) dan target (y)
X = df.drop('cluster', axis=1)
y = df['cluster']

# Bagi data menjadi data latih dan data uji dengan rasio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menampilkan ukuran data latih dan data uji
print("Ukuran data latih:", X_train.shape, y_train.shape)
print("Ukuran data uji:", X_test.shape, y_test.shape)

Ukuran data latih: (2144, 9) (2144,)
Ukuran data uji: (536, 9) (536,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [ ]:
categorical_features = df.select_dtypes(include=['object']).columns

label_encoders = {}
for feature in categorical_features:
    label_encoders[feature] = LabelEncoder()
    df[feature] = label_encoders[feature].fit_transform(df[feature])

X = df.drop('cluster', axis=1)
y = df['cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
svm_classifier = SVC(random_state=42)
svm_classifier.fit(X_train, y_train)

y_pred_svm = svm_classifier.predict(X_test)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [ ]:
#Evaluasi Performa Random Forest
y_train_pred = rf_classifier.predict(X_train)
y_test_pred = rf_classifier.predict(X_test)
y_pred_dt = rf_classifier.predict(X_test)

acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)
f1_train = f1_score(y_train, y_train_pred, average='weighted')
f1_test = f1_score(y_test, y_test_pred, average='weighted')
precision_test = precision_score(y_test, y_test_pred, average='weighted')
recall_test = recall_score(y_test, y_test_pred, average='weighted')

print("Precision:", precision_test)
print("Recall:", recall_test)
print("Random Forest Classifier:")
print("Training Accuracy:", acc_train)
print("Testing Accuracy:", acc_test)
print("F1 Score Training:", f1_train)
print("F1 Score Testing:", f1_test)
print(confusion_matrix(y_test,y_pred_dt))

Precision: 1.0
Recall: 1.0
Random Forest Classifier:
Training Accuracy: 1.0
Testing Accuracy: 1.0
F1 Score Training: 1.0
F1 Score Testing: 1.0
[[183   0]
 [  0 353]]


In [ ]:
# Evaluasi Performa model SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Akurasi SVM: {accuracy_svm}")

confusion_svm = confusion_matrix(y_test, y_pred_svm)
print(f"Confusion Matrix SVM:\n{confusion_svm}")

precision_svm = precision_score(y_test, y_pred_svm, average='weighted')
recall_svm = recall_score(y_test, y_pred_svm, average='weighted')
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')

print(f"Precision SVM: {precision_svm}")
print(f"Recall SVM: {recall_svm}")
print(f"F1-score SVM: {f1_svm}")

Akurasi SVM: 0.9738805970149254
Confusion Matrix SVM:
[[176   7]
 [  7 346]]
Precision SVM: 0.9738805970149254
Recall SVM: 0.9738805970149254
F1-score SVM: 0.9738805970149254


Disitu terlihat bahwa F1 score dan yang lainnya lebih bagus ketika menggunakan algoritma Random Forest yang hasilnya itu 1.0 dari pada SVM yang 0.97. Itu menandakan bahwa Random Forest lebih baik dari SVM untuk data ini

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_dist = {
    'C': uniform(0.1, 10),
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 7)),
    'degree': randint(2, 5)
}


# Buat objek RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=SVC(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

# Latih model dengan RandomizedSearchCV
random_search.fit(X_train, y_train)


# Cetak hasil terbaik
print("Parameter terbaik:", random_search.best_params_)
print("Akurasi terbaik:", random_search.best_score_)

Parameter terbaik: {'C': 3.845401188473625, 'degree': 2, 'gamma': 100.0, 'kernel': 'linear'}
Akurasi terbaik: 1.0


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
best_params = random_search.best_params_
best_svm_model = SVC(**best_params, random_state=42)
best_svm_model.fit(X_train, y_train)

y_pred_best_svm = best_svm_model.predict(X_test)

accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
print(f"Akurasi SVM dengan parameter terbaik: {accuracy_best_svm}")

confusion_best_svm = confusion_matrix(y_test, y_pred_best_svm)
print(f"Confusion Matrix SVM dengan parameter terbaik:\n{confusion_best_svm}")

precision_best_svm = precision_score(y_test, y_pred_best_svm, average='weighted')
recall_best_svm = recall_score(y_test, y_pred_best_svm, average='weighted')
f1_best_svm = f1_score(y_test, y_pred_best_svm, average='weighted')

print(f"Precision SVM dengan parameter terbaik: {precision_best_svm}")
print(f"Recall SVM dengan parameter terbaik: {recall_best_svm}")
print(f"F1-score SVM dengan parameter terbaik: {f1_best_svm}")

Akurasi SVM dengan parameter terbaik: 1.0
Confusion Matrix SVM dengan parameter terbaik:
[[183   0]
 [  0 353]]
Precision SVM dengan parameter terbaik: 1.0
Recall SVM dengan parameter terbaik: 1.0
F1-score SVM dengan parameter terbaik: 1.0


Naik dari 0.97 ke 1.0

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

1. Karena kita hanya melakukan hyperparameter tuning di SVM yang belum 1.0, terlihat jelas ada peningkatan walau ngak signifikan dari sekita 0.97 ke 1.0.
2. Untuk Random Forest ngak di hyperparameter tuning karena memang sudah 1.0 jadi tidak perlu di ubah lagi.
3. Walau pada awalnya lebih bagus Random Forest tapi setelah di hyperparameter SVM juga bisa ikut menyusul Random forest.